# Step 1 - Preparatory processes

Log into accre (via a terminal window)

In [ ]:
ssh middlepg@login.accre.vanderbilt.edu

##### Step 1.1 - Check out specific versions of various toolboxes

_Goal_:  
Control the version of the toolboxes used; either update to most recent (present experiment), or update to correct commit (past experiment).

_Instructions_:  
Paste code into terminal. Make sure to clone versions of these toolboxes into your own home directory and refer to those paths.

_Output_:  
None

###IN ACCRE, Checkout the right commit of source codes (matlab and slurm scripts)

####If running a present experiment, update to the latest code:

In [2]:
# Check out analysis code
cd /scratch/middlepg/perceptualchoice_stop_model/


git pull

# or

git fetch --all
# then
git reset --hard origin/master


SyntaxError: invalid syntax (<ipython-input-2-7134f1a007c8>, line 2)

##### Step 1.2 - Add paths to MATLAB's search path

_Goal_:  
Ensure access to SAM toolbox and job code

_Instructions_:  
Paste code into MATLAB

_Output_:  
None

In [4]:
%%matlab
accreRoot = '/gpfs22';
accreHome = '/home/middlepg';
accreScratch = '/scratch/middlepg';
if isdir(fullfile(accreScratch))
    matRoot = fullfile(accreRoot,accreHome,'m-files');
    modelRoot = fullfile(accreScratch,'perceptualchoice_stop_model');
    environ = 'accre';
else
    matRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/schalllab';
    modelRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model';
    environ = 'local';
end

addpath(genpath(fullfile(matRoot,'sam')));
addpath(genpath(fullfile(matRoot,'matlab_code_bbz')));
addpath(genpath(fullfile(matRoot,'matlab_file_exchange_tools')));
addpath(genpath(fullfile(matRoot,'cmtb')));
addpath(genpath(fullfile(modelRoot,'src/code/2017-03-28')));

cd(fullfile(modelRoot,'src/code/2017-03-28/matlab'))


ERROR: Cell magic `%%matlab` not found.


##### Step 1.3 - Copy previous experiment's preprocessed data

In [ ]:
%%matlab
rawDataDir          = fullfile(modelRoot,'data/raw/');
preprocDataDir      = fullfile(modelRoot,'data/2017-03-06/preproc01/');

preprocDataDirCurrent      = fullfile(modelRoot,'data/2017-03-28/preproc01/');


subject             = 'subj02';
sessions            = 'data_subj02.mat';
mkdir(fullfile(preprocDataDirCurrent,subject))
copyfile(fullfile(preprocDataDir,subject,sessions),fullfile(preprocDataDirCurrent,subject,sessions))


##### Step 1.4 - Copy previous experiment's random number generator seed

In [ ]:
%%matlab
preprocDataDir      = fullfile(modelRoot,'data/2017-03-06/preproc01/');
preprocDataDirCurrent      = fullfile(modelRoot,'data/2017-03-28/preproc01/');

copyfile(fullfile(preprocDataDir,'rngStruct.mat'), fullfile(preprocDataDirCurrent,'rngStruct.mat'))


## Step 2 - Fits


##### Step 5.1 - Build the general SAM structure


_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_allTrials.mat`

In [ ]:
%%matlab
subj                    = 2;
dt                      = 1;
trialVar                = true;
optimScope              = 'all';
choiceMech              = {'race','ffi','li'};
stopMech                = {'li'};
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2017-03-28/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_general(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},rngStruct,fileStr);
    end
end


##### Step 5.2 - Extract best-fitting GO and STOP parameters

In [ ]:
%%matlab
subj                    = 2;
trialVar                = true;
dt                      = 1;
simScope                = 'all';
stopMech                = 'li';
doPlot                  = true;
doSave                  = true;
doStartParCluster       = false;

fileStrStart.result            = strcat(modelRoot,'/results/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStrStart.root            = strcat(modelRoot,'/data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');
fileStrStart.fitRoot         = strcat(modelRoot,'/data/2017-03-06/preproc01/subj%.2d/dt%d/%s/%s/');

fileStrFit.root            = strcat(modelRoot,'/data/2017-03-28/preproc01/subj%.2d/dt%d/%s/%s/');


for iSubj = subj
    switch iSubj

        case 2
            modelStartFrom = [352 352 352 352 352 352];
            choiceStartFrom = {'race','race','race','race','race','race'};
            modelToFit = [352 478 352 478 352 478];
            choiceToFit = {'race','race','li','li','ffi','ffi'};
    end       




    for iFit = 2 : 2%length(modelStartFrom)


        p = find_best_params(iSubj,trialVar,simScope,choiceStartFrom{iFit},stopMech,dt,modelStartFrom(iFit),fileStrStart)

            % Get corresponding row of parameters
            ind = (p.subject == iSubj & strcmp(p.unit, 'go'));


        switch modelStartFrom(iFit)
            case 79
                switch modelToFit(iFit)
                    case 79
                        % Do nothing
                end
            case 352
                switch modelToFit(iFit)
                    case 352
                        % Do nothing
                    case 478
                        p.z0{ind} = repmat(p.z0{ind},1,2);
                end
        end
        
        x0Base = table2cell(p(:,4:end))


        % Specify the initial set of parameter and save/plot, if required
        %job_spec_x0base(iSubj,dt,trialVar,simScope,choiceStartFrom{iFit},stopMech,modelStartFrom(iFit),fileStrStart,x0Base,doPlot,doSave,doStartParCluster);
        job_spec_x0base(iSubj,dt,trialVar,simScope,choiceToFit{iFit},stopMech,modelToFit(iFit),fileStrFit,x0Base,doPlot,doSave,doStartParCluster);

    end
end


##### Step 5.3 - Build model-specific SAM structure


_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_allTrials_model%.3d.mat`

In [ ]:
%%matlab
subj                    = 2;
trialVar                = true;
dt                      = 1;
optimScope                = 'all';
stopMech                = 'li';
doStartParCluster       = false;
fileStrStart.root            = strcat(modelRoot,'/data/2017-03-28/preproc01/subj%.2d/dt%d/%s/%s/');

for iSubj = subj
    switch iSubj
        case 2
            modelToFit = [478 352 478 352 478];
            choiceToFit = {'race','li','li','ffi','ffi'};
    end       

    for iFit = 1 : length(modelToFit)

        job_spec_sam_model(iSubj,dt,trialVar,optimScope,choiceToFit{iFit},stopMech,modelToFit(iFit),fileStrStart);

    end
end


##### Step 5.4 - Automatic optimization of GO and STOP parameters


_Goal_:  
Find best-fitting GO and STOP parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.sh` on ACCRE

_Output_:  
`finalLog_allTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_allpTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_allTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

In [ ]:
%%system
scratch
cd perceptualchoice_stop_model/src/code/2017-03-28/sbatch

./exe_job_optim_x_All_xena_352.txt
./exe_job_optim_x_All_xena_478.txt


##### Step 5.5 - Merge optimization files


_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_allTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = [1:2];
dt          = 1;
trialVar    = true;
optimScope  = 'all';
stopMech    = {'li'};
fileStr.root  = fullfile(modelRoot,'data/2017-03-28/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';


for iSubj = subj
    switch iSubj
        case 1
            modelToFit = [79 79 478];
            choiceToFit = {'ffi','li','li'};

        case 2
            modelToFit = [79 79 352 352 478 478 478];
            choiceToFit = {'ffi','li','li','ffi','race','li','ffi'};
    end       

    for iFit = 1 : length(modelToFit)

            job_merge_fvalandbestx(iSubj,dt,trialVar,optimScope,choiceToFit{iFit},stopMech{1},modelToFit(iFit),fileStr);

    end
end



##### Step 5.2 - Extract best-fitting GO and STOP parameters at 1 ms resolution

In [ ]:
%%system
mkdir /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj01/dt1/
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj01/dt1/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj01/dt1/

mkdir /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj02/dt1/
scp -r middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj02/dt1/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2017-03-28/preproc01/subj02/dt1/
        
